<a href="https://colab.research.google.com/github/drmartins2/EDIT_DE/blob/main/spark/examples/06-write_partitioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Write
- .write
- .format (parquet, csv, json)
- options
- spark.sql.sources.partitionOverwriteMode dynamic

# Write Mode
- overwrite - The overwrite mode is used to overwrite the existing file, alternatively, you can use SaveMode.Overwrite
- append - To add the data to the existing file, alternatively, you can use SaveMode.Append
- ignore - Ignores write operation when the file already exists, alternatively, you can use SaveMode.Ignore.
- errorifexists or error - This is a default option when the file already exists, it returns an error, alternatively, you can use SaveMode.ErrorIfExists.

# Partitioning
Process to organize the data into multiple chunks based on some criteria.
Partitions are organized in sub-folders.
Partitioning improves performance in Spark.

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').getOrCreate()

# Preparing data

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.2 MB/s eta 0:00:00


In [3]:
from faker import Faker
from datetime import datetime

fake = Faker()

users = []
for _ in range(50):
    user = {
        'date': fake.date_time_between_dates(datetime(2024, 5, 1), datetime(2024, 5, 5)),
        'name': fake.name(),
        'address': fake.address(),
        'email': fake.email(),
        'dob': fake.date_of_birth(),
        'phone': fake.phone_number()
    }
    users.append(user)

df = spark.createDataFrame(users)

df.show(10, False)


+-------------------------------------------------------+--------------------------+----------+-------------------------+---------------+----------------------+
|address                                                |date                      |dob       |email                    |name           |phone                 |
+-------------------------------------------------------+--------------------------+----------+-------------------------+---------------+----------------------+
|38065 Robert Street Apt. 753\nGallagherton, TN 45397   |2024-05-04 15:36:51.796586|1909-01-15|timothylucas@example.net |Megan Porter   |323-593-8235x38167    |
|3447 Keith Estate Suite 019\nGaryfurt, KY 10475        |2024-05-02 23:21:58.681493|2004-08-05|rschultz@example.net     |Janice Lane    |272-885-8870x8138     |
|736 Ho Union Apt. 862\nHermanborough, DE 82724         |2024-05-01 19:12:52.895824|1985-10-30|ulucas@example.com       |Corey Perez Jr.|(900)650-6425x89058   |
|36048 Charles Summit\nDustinland,

# Writing as PARQUET



In [4]:
# Writing as PARQUET with no partitions

path = "/content/write_partitioning/parquet_no_partitions"

df.write.mode("overwrite").format("parquet").save(path)

!ls /content/write_partitioning/parquet_no_partitions

spark.read.format("parquet").load(path).count()

part-00000-394260dc-04f0-4d95-8960-8947ae1a8a74-c000.snappy.parquet  _SUCCESS


50

In [7]:
# Remove the created partition folder 20240502
!rm -rf /content/write_partitioning/parquet_with_partitions/date_part\=20240502

In [8]:
# Writing as PARQUET with partitions
from pyspark.sql.functions import *

path = "/content/write_partitioning/parquet_with_partitions"

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd"))

spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic") # enable dynamic partition overwrite - only overwrites partitions that are coming in the dataframe

(df.where("date_part = '20240502'")
 .write
 .mode("overwrite")                                               # overwrites the entire path with the new data
 .partitionBy("date_part")                                        # partition the data by column - creates sub-folders for each partition
 .format("parquet")                                               # format of output
 .save(path))                                                     # path

!ls /content/write_partitioning/parquet_with_partitions

spark.read.format("parquet").load(path).count()

'date_part=20240501'  'date_part=20240502'  'date_part=20240503'  'date_part=20240504'


50

Side note:
If several small files are created, the OPTIMIZE function can be used to merge all partioned small files into one (with a max size of 200mb each)

https://docs.delta.io/latest/optimizations-oss.html#language-sql

In [9]:
# Checking single partition
spark.read.parquet("/content/write_partitioning/parquet_with_partitions/date_part=20240502").show()

+--------------------+--------------------+----------+--------------------+-----------------+--------------------+
|             address|                date|       dob|               email|             name|               phone|
+--------------------+--------------------+----------+--------------------+-----------------+--------------------+
|3447 Keith Estate...|2024-05-02 23:21:...|2004-08-05|rschultz@example.net|      Janice Lane|   272-885-8870x8138|
|Unit 4482 Box 437...|2024-05-02 03:54:...|2023-12-18|heatherabbott@exa...|       Amy Wilson|   603.679.7271x6952|
|8885 Vargas Cross...|2024-05-02 06:03:...|1948-08-24|hicksstacy@exampl...|     Darren Jones|  (783)984-3674x5555|
|2809 Palmer Way S...|2024-05-02 22:22:...|1952-06-28|   vrice@example.com|     Eric Whitney|+1-924-352-3176x1...|
|631 Aaron Parkway...|2024-05-02 19:45:...|1993-12-29| david20@example.org|   David Phillips|        347-273-4411|
|PSC 2498, Box 994...|2024-05-02 12:59:...|2013-12-22| linda43@example.net|     

# Writing as CSV

https://spark.apache.org/docs/3.5.1/sql-data-sources-csv.html

In [10]:
df.count()

50

In [11]:
path = "/content/write_partitioning/csv_no_partitioning/"

# write as csv
(df
  .write
  .format("csv")
  .mode("overwrite")
  .option("delimiter", "|")
  .option("header", True)
  .save(path))

# listing files in the folder
!ls /content/write_partitioning/csv_no_partitioning/

# read as csv
(spark
  .read
  .options(sep="|", multiLine=True, header=True)
  .csv(path)
  .count())

part-00000-0301f8ef-33b6-4339-90c4-2663ff4599f1-c000.csv  _SUCCESS


50

# Writing as JSON

https://spark.apache.org/docs/3.5.1/sql-data-sources-json.html

In [13]:
path = "/content/write_partitioning/json_no_partitioning/"

# write as json
(df
.write
.mode("overwrite")
.format("json")
.save(path))

# listing files in the folder
!ls /content/write_partitioning/json_no_partitioning/

# read as json
(spark
  .read
  .json(path)
  .count())

part-00000-9941d8aa-29b2-41f1-933b-57a850774cdb-c000.json  _SUCCESS


50

In [14]:
# reading json as text
spark.read.text(path).show(10, False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                 |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"address":"38065 Robert Street Apt. 753\nGallagherton, TN 45397","date":"2024-05-04T15:36:51.796Z","dob":"1909-01-15","email":"timothylucas@example.net","name":"Megan Porter","phone":"323-593-8235x38167","date_part":"20240504"}  |
|{"address":"3447 Keith Estate Suite 019\nGaryfurt, KY 10475","date"

In [15]:
# reading json as text
spark.read.json(path).show(10, False)

+-------------------------------------------------------+------------------------+---------+----------+-------------------------+---------------+----------------------+
|address                                                |date                    |date_part|dob       |email                    |name           |phone                 |
+-------------------------------------------------------+------------------------+---------+----------+-------------------------+---------------+----------------------+
|38065 Robert Street Apt. 753\nGallagherton, TN 45397   |2024-05-04T15:36:51.796Z|20240504 |1909-01-15|timothylucas@example.net |Megan Porter   |323-593-8235x38167    |
|3447 Keith Estate Suite 019\nGaryfurt, KY 10475        |2024-05-02T23:21:58.681Z|20240502 |2004-08-05|rschultz@example.net     |Janice Lane    |272-885-8870x8138     |
|736 Ho Union Apt. 862\nHermanborough, DE 82724         |2024-05-01T19:12:52.895Z|20240501 |1985-10-30|ulucas@example.com       |Corey Perez Jr.|(900)650-6

In [18]:
# partition json data + saveAsTable

# Creating partition column
df = df.withColumn("date_part", date_format(col("date"), "yyyyMMdd"))

# write as json
(df.write
  .partitionBy("date_part")
  .mode("overwrite")
  .format("json")
  .saveAsTable("tbl_json_part"))

# read as json
print(spark.table("tbl_json_part").count())

# show created table
spark.sql("show tables from default").show()

# read as json
spark.sql("show partitions tbl_json_part").show()

50
+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|  default|tbl_json_part|      false|
+---------+-------------+-----------+

+------------------+
|         partition|
+------------------+
|date_part=20240501|
|date_part=20240502|
|date_part=20240503|
|date_part=20240504|
+------------------+



# Append Mode

In [19]:
# Writing as PARQUET with APPEND

path = "/content/write_partitioning/parquet_append"

df.write.mode("append").format("parquet").save(path)

!ls /content/write_partitioning/parquet_append

spark.read.format("parquet").load(path).count()

part-00000-72f41d21-4505-4134-b753-0e1f6b1ba9e1-c000.snappy.parquet  _SUCCESS


50